In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Pass the csv files into variables.

In [ ]:
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
item_categories = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
sales = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv',  parse_dates=['date'], 
                    dtype={'date': 'str', 'date_block_num': 'int32', 'shop_id': 'int32','item_id': 'int32', 'item_price': 'float32', 'item_cnt_day': 'int32'})
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

We need to reduce data usage. We can do this with the following code.

In [ ]:
def datatype_downcast(df):
    float_col= [i for i in df if df[i].dtype == 'float64']
    int_col = [i for i in df if df[i].dtype in ['int64','int32']]
    df[float_col] = df[float_col].astype(np.float32)
    df[int_col] = df[int_col].astype(np.int16)
    return df

sales = datatype_downcast(sales)

Let's look at our data to get an idea of what they contain.

In [ ]:
items.head()

In [ ]:
item_categories.head()

We can see that there are similarities between the item categories variable and items. We will have to join these together later.

In [ ]:
shops.head()

In [ ]:
sales.head()

We have a few negative values in item count per day and item price. These variables cannot be negative, so we will treat them.

In [ ]:
train_data = sales.join(items, on = 'item_id', rsuffix='_').join(item_categories, on = 'item_category_id', rsuffix = '_').join(shops, on = 'shop_id', rsuffix = '_').drop(['item_id_', 'shop_id_', 'item_category_id_', 'item_name'], axis=1)
train_data['item_cnt_day'] = np.abs(train_data['item_cnt_day'])
train_data['item_price'] = np.abs(train_data['item_price'])
train_data.head()

We can organize this dataset better. Let's sum the item count per day for each day for easier analysis.

In [ ]:
import datetime as dt

train_data['date'] = pd.to_datetime(train_data['date'], format='%d.%m.%Y')
train_data['day'] = train_data['date'].dt.day
train_data['month'] = train_data['date'].dt.month
train_data['year'] = train_data['date'].dt.year

train_data.drop('date', axis=1, inplace=True)

train_data

In [ ]:
monthly_sales = train_data[["month", "date_block_num", "shop_id", "item_id", "item_price", "item_cnt_day", "item_category_id", "year"]].groupby(['date_block_num',"shop_id", "item_id", "item_category_id"]).agg({"item_price":"mean","item_cnt_day":"sum","month":"min", "year":"min"}).reset_index()

In [ ]:
monthly_sales.rename(columns={"item_cnt_day":"item_cnt_month"},inplace=True)
monthly_sales.head()

# Data Visualization

Visualization is always vital in analysis. Let's try to plot a few graphs for easier comprehension.

In [ ]:
import seaborn as sns
sns.heatmap(monthly_sales.corr())

In [ ]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
rcParams['figure.figsize'] = 20,20
sns.barplot(x = 'item_category_id', y = 'item_id', data = monthly_sales)
plt.xlabel('Item Category', fontsize = 25)
plt.ylabel('Item', fontsize = 25)
plt.show()

In [ ]:
item_categories.iloc[83]

In [ ]:
item_categories.iloc[45]

Элементы питания is the most popular item category with Книги - Аудиокниги 1С being the least.

In [ ]:
grouped_monthly_sales = monthly_sales.groupby(['date_block_num'])['item_cnt_month'].sum()
grouped_monthly_sales.plot()

In [ ]:
sns.countplot(monthly_sales['shop_id'])
plt.show()

In [ ]:
shops.iloc[31]

Москва ТЦ "Семеновский" is the most popular shop, making the most sales.

# Data Cleaning

We have to look for any NA values and treat them accordingly before working on our dataset. Let's take a look and see if there are any.

In [ ]:
train_data.isnull().sum()

There are no NA values in this dataset. However we can still clean the existing values by eliminating outliers.

In [ ]:
monthly_sales = monthly_sales[monthly_sales['item_cnt_month']>0]
monthly_sales = monthly_sales[monthly_sales['item_price'] < 100000]
monthly_sales = monthly_sales[monthly_sales['item_cnt_month'] < 1200]

In [ ]:
monthly_sales.head()

# Encoding

In [ ]:
shop_mean = monthly_sales.groupby(['shop_id']).agg({'item_cnt_month': ['mean']})
shop_mean.columns = ['shop_mean']
shop_mean.reset_index(inplace=True)

In [ ]:
item_mean = monthly_sales.groupby(['item_id']).agg({'item_cnt_month': ['mean']})
item_mean.columns = ['item_mean']
item_mean.reset_index(inplace=True)

In [ ]:
shop_item_mean = monthly_sales.groupby(['shop_id', 'item_id']).agg({'item_cnt_month': ['mean']})
shop_item_mean.columns = ['shop_item_mean']
shop_item_mean.reset_index(inplace = True)

In [ ]:
month_mean = monthly_sales.groupby(['month']).agg({'item_cnt_month': ['mean']})
month_mean.columns = ['month_mean']
month_mean.reset_index(inplace=True)

In [ ]:
year_mean = monthly_sales.groupby(['year']).agg({'item_cnt_month': ['mean']})
year_mean.columns = ['year_mean']
year_mean.reset_index(inplace = True)

In [ ]:
monthly_sales = pd.merge(monthly_sales, shop_mean, on=['shop_id'], how='left')
monthly_sales = pd.merge(monthly_sales, item_mean, on=['item_id'], how='left')
monthly_sales = pd.merge(monthly_sales, shop_item_mean, on = ['shop_id', 'item_id'], how = 'left')
monthly_sales = pd.merge(monthly_sales, month_mean, on=['month'], how='left')
monthly_sales = pd.merge(monthly_sales, year_mean, on = ['year'], how = 'left')

In [ ]:
test.head()

Observing our test dataset, we have shop ids and item ids. For more efficient analysis, we should take the unique values can compare them to what we have in our training dataset.

In [ ]:
test_shop_ids = test['shop_id'].unique()
test_item_ids = test['item_id'].unique()
train_in_test = monthly_sales[monthly_sales['shop_id'].isin(test_shop_ids)]
train_in_test = monthly_sales[monthly_sales['item_id'].isin(test_item_ids)]

In [ ]:
train_in_test.head()

In [ ]:
train_in_test.shape

# Modeling

We need to separate the target variable from the rest of our data before we start modeling.

In [ ]:
y_data = train_in_test['item_cnt_month']
x_data = train_in_test.drop('item_cnt_month', axis = 1)
x_data.head()

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
X = preprocessing.StandardScaler().fit(x_data).transform(x_data)
x_train, x_test, y_train, y_test = train_test_split(X, y_data, test_size = 0.25, random_state = 42)

### Random Forest Regression

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(n_estimators = 135, max_depth=20, random_state=42, n_jobs= 16, criterion='mse')
forest_reg.fit(x_train, y_train)
score = forest_reg.score(x_test, y_test)
print(score)

The random forest regressor performs best on our data, successfully predicting the test data at a rate of 86.6%

In [ ]:
print('Model RMSE...')
y_pred_test = forest_reg.predict(x_test)
rmse_forest = mean_squared_error(y_test, y_pred_test, squared = False)
print(rmse_forest)

### XGBoost

In [ ]:
import xgboost as xg
xgb_regressor = xg.XGBRegressor(n_estimators = 150, learning_rate = 0.2, random_state = 42, n_jobs = 16)
xgb_regressor.fit(x_train, y_train)
score = xgb_regressor.score(x_test, y_test)
print(score)

XGBoost has an 86.1% success rate in predicting our data using these parameters.

In [ ]:
print('Model RMSE...')
y_pred_valid_xgb = xgb_regressor.predict(x_test)
rmse_xgb = mean_squared_error(y_test, y_pred_valid_xgb, squared=False)
print(rmse_xgb)

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
LR = LinearRegression()
LR.fit(x_train, y_train)
score = LR.score(x_test, y_test)
print(score)

In [ ]:
LR_pred = LR.predict(x_test)
rmse_LR = mean_squared_error(y_test, LR_pred, squared = False)
print(rmse_LR)

# Data Joining

Since there are only two parameters in the test set, and the model I built has a lot more than that, it will be easier if we join our train set with our test set.

In [ ]:
sales_train_new = train_in_test.pivot_table(index= ['shop_id', 'item_id'], columns = 'date_block_num', values = 'item_cnt_month', fill_value = 0).reset_index()
sales_train_new = pd.merge(test,sales_train_new,on = ['shop_id', 'item_id'],how = 'left')
sales_train_new.fillna(0, inplace = True)
sales_train_new.head().T

In [ ]:
sales_train_new.shape

Working with our date block number, we are going to predict the last possible month number.

In [ ]:
X_train = sales_train_new.drop(33, axis=1)
y_train1 = sales_train_new[32]
X_test = sales_train_new.drop(0, axis=1)
y_test1 = sales_train_new[33]

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train1.shape)
print(y_test1.shape)

I am using the best model from earlier with the same parameters to ensure uniformity.

In [ ]:
from sklearn.metrics import r2_score
forest_reg1 = RandomForestRegressor(n_estimators = 135, max_depth=20, random_state=42, n_jobs= 16, criterion='mse')
forest_reg1.fit(X_train, y_train1)
y_pred_train = forest_reg1.predict(X_train)
y_pred_test = forest_reg1.predict(X_test)
r2_train_lr = r2_score(y_train1, y_pred_train)
print(r2_train_lr)

We get a very high R-squared value of 0.9951 in this one. Our model is excellent at predicting the last month.

In [ ]:
forest_reg1.predict(X_test).clip(0,20)

In [ ]:
submission = pd.DataFrame({'ID':X_test.index,'item_cnt_month':forest_reg1.predict(X_test)})

In [ ]:
submission.shape

In [ ]:
submission.to_csv('submission.csv', index=False)